# 뉴스 요약봇 만들기
-----
### 프로젝트 - 뉴스기사 요약해보기

## 실습목표

>* Extractive/Abstractive summarization 이해하기
>* 단어장 크기를 줄이는 다양한 text normalization 적용해보기
>* seq2seq의 성능을 Up시키는 Attention Mechanism 적용하기

## 순서
> **Step1.** 데이터 수집하기<br>
**Step2.** 데이터 전처리하기 (추상적 요약)<br>
**Step3.** 어텐션 메커니즘 사용하기 (추상적 요약)<br>
**Step4.** 실제 결과와 요약문 비교하기 (추상적 요약)<br>
**Step5.** Summa을 이용해서 추출적 요약해보기

## 용어정리
---

**✓ 텍스트 요약(Text Summarization)**이란?
>* 긴 길이의 문서(Document) 원문을 핵심 주제만으로 구성된 짧은 요약(Summary) 문장들로 변환하는 것<br>
>* 이때 중요한 것은 요약 전후에 정보 손실 발생이 최소화되어야 한다는 점!(정보를 압축하는 과정과 같음)
>* 크게 추출적 요약(Extractive Summarization)과 추상적 요약(Abstractive Summarization)의 두가지 접근으로 나누어볼 수 있음

>* **추출적 요약(Extractive Summarization)**이란?
>> * 원문에서 문장들을 추출해서 요약하는 방식<br>
>> * 딥 러닝보다는 주로 전통적인 머신 러닝 방식에 속하는 텍스트랭크(TextRank)와 같은 알고리즘을 사용해서 이 방법을 사용
>> * 핵심문장인지를 판별한다는 점에서 문장 분류(Text Classification) 문제로 볼 수 있을 것
>> * 가장 대표적인 것이 네이버 뉴스 서비스에 있는 요약봇 기능
>* **추상적 요약(Abstractive Summarization)**이란?
>> * 원문으로부터 내용이 요약된 새로운 문장을 생성<br>
>> * 자연어 생성(Natural Language Generation, NLG)의 영역인 셈
>> * 이때 중요한 것은 요약 전후에 정보 손실 발생이 최소화되어야 한다는 점!(정보를 압축하는 과정과 같음)

**✓ NLTK**란?
>* **N**atural **L**anguage **T**oolkit의 축약어
>* 이 NLTK에는 I, my, me, over, 조사, 접미사와 같이 문장에는 자주 등장
>* 의미를 분석하고 요약하는데는 거의 의미가 없는 100여개의 불용어가 미리 정리되어 있음!

우리는 NLTK를 이용해 다운로드받은 리뷰 파일에서 불용어를 제거하는 작업을 진행할 예정입니다.<br>

**✓ seq2seq(sequence-to-sequence)**란?
>* 인코더와 디코더의 구조로 구성된 딥러닝 아키텍처
>* 두 개의 RNN 아키텍처를 사용하여 입력 시퀀스로부터 출력 시퀀스를 생성해내는 자연어 생성 모델
>* 주로 뉴럴 기계번역에 사용 ➡️ 원문을 요약문으로 번역 과정과 비슷

**✓ 인공 신경망으로 텍스트 요약 훈련시키기**
1. seq2seq를 사용합니다.
2. RNN 계열 중 LSTM을 사용하므로 hidden state뿐만 아니라 cell state도 사용해야 합니다.
3. 디코더의 예측 시퀀스에는 시작 토큰 SOS와 예측 토큰 EOS를 시퀀스의 앞, 뒤로 붙입니다.
4. seq2seq를 구동시키면 디코더는 시작 토큰을 입력받아 예측을 시작합니다.
5. seq2seq 기본 모델과 달리, 어텐션 메커니즘을 이용해 인코더의 hidden state의 중요도를 취합한 컨텍스트 벡터를 디코더 스텝별로 계산합니다.
6. 계산된 컨텍스트 벡터를 이용해서 디코더는 다음 등장할 단어를 예측합니다.

### <span style="color:green">Step1. 데이터 수집하기</span>
---
**✓ 그럼 데이터를 준비해 볼까요?**

오늘 사용할 패키지를 불러옵니다.<br>NTLK와 NLTK 데이터셋이 설치되어있지 않은 환경이라면 우선 NLTK를 설치하고 NTLK의 데이터셋을 다운로드해주세요.<br>터미널에서 **`pip install [패키지명]`**을 사용해 필요한 패키지를 설치해 주세요.<br>아래 명령어를 복사해서 설치해도 됩니다.

> $ pip install nltk

BeautifulSoup 라이브러리도 설치하지 않았다면 설치해주세요.

> $ pip install beautifulsoup4

nltk 패키지에서 불용어 사전을 다운로드 받고, 데이터 전처리를 위한 나머지 패키지도 함께 불러옵니다.

In [5]:
import nltk  # 영어 기호, 통계, 자연어 처리를 위한 라이브러리
nltk.download('stopwords')  # NLTK의 불용어(stopwords)를 사용
from nltk.corpus import stopwords

import numpy as np  # C언어로 구현된 파이썬 라이브러리, pandas와 matplotlib의 기반으로 사용
import pandas as pd  #  데이타 분석(Data Analysis)을 위해 사용(1차원 자료구조인 Series, 2차원 자료구조인 DataFrame, 그리고 3차원 자료구조인 Panel을 지원)
import os  # 운영 체제와 상호 작용하기 위한 수십 가지 함수들을 제공
import re  # 정규 표현식을 사용하고자 할 때 사용(regex의 약자)
import matplotlib.pyplot as plt  # 라인 플롯, 바 차트, 파이차트, 히스토그램, Box Plot, Scatter Plot 등을 비롯하여 다양한 차트와 플롯 스타일을 지원
from bs4 import BeautifulSoup  # 문서를 파싱하는데 사용하는 패키지
from tensorflow.keras.preprocessing.text import Tokenizer  # 케라스(Keras)의 토큰화 함수를 이용해 텍스트 전처리
from tensorflow.keras.preprocessing.sequence import pad_sequences  # padding을 사용하여 길이를 일정하게 만들며 sequences에는 길이가 다른 데이터가 들어감
import urllib.request  # URL 열기 및 읽기

[nltk_data] Downloading package stopwords to /home/aiffel-
[nltk_data]     dj49/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

data.sample(10)

,headlines,text
4197,Panel to conduct survey for sub-categorisation...,A commission examining the sub-categorisation ...
92225,Minister Babul Supriyo shares wrong pic of Raj...,Union Minister Babul Supriyo has shared a phot...
69216,Domestic worker arrested for burglary at Myntr...,The Bengaluru Police has arrested Myntra CEO A...
6429,B'luru home services platform Housejoy raises ...,Bengaluru-based online home services platform ...
56295,Darjeeling toy train resumes operations after ...,"The Darjeeling toy train, also known as the Da..."
5739,Saturn losing its rings at 'worst-case-scenari...,A new NASA research has confirmed Saturn is lo...
19544,Ex-Andhra CM's son could've survived had he wo...,"Former Andhra Pradesh CM NTR Rao's son, Nandam..."
48403,"What is Vantablack, used to make 'world's dark...","Dubbed the ""world's blackest black"", Vantablac..."
10878,11 properties of Nirav Modi worth Ã¢ÂÂ¹56 cr ...,The Enforcement Directorate (ED) on Tuesday sa...
74889,"I-T Dept summons Lalu's daughter Misa Bharti, ...",The Income Tax Department on Thursday asked RJ...


이 데이터는 기사의 본문에 해당되는 text와 headlines 두 가지 열로 구성되어져 있습니다.

추상적 요약을 하는 경우에는 text를 본문, headlines를 이미 요약된 데이터로 삼아서 모델을 학습할 수 있어요. 추출적 요약을 하는 경우에는 오직 text열만을 사용하세요.

In [8]:
data = data[['text','headlines']]
data.head()

#랜덤한 5개 샘플 출력
data.sample(5)

,text,headlines
59639,Indian pacer Bhuvneshwar Kumar tied the knot w...,Bhuvneshwar Kumar ties the knot with childhood...
81604,US First Lady Melania Trump was sent in to end...,Melania tried to end 'overtime' meeting betwee...
23147,"Marie Laguerre, who was attacked by her harass...",French woman hit by harasser starts online for...
53552,Pakistan's PM Shahid Abbasi has claimed his co...,We received very insignificant amount of US ai...
3387,"Ayushmann Khurrana's wife Tahira Kashyap, who ...",Can't keep calm: Ayushmann's wife Tahira on co...


**Error]** **<span style="color:Orange">에러명</span>**: 사유

### <span style="color:green">Step2. 데이터 전처리하기 (추상적 요약)</span>
---
이제 데이터를 불러왔으니 전처리를 진행해 볼게요. 빈칸으로 존재하는 null 데이터, 의미는 같지만 다른 식으로 작성된 글 같은 중복 항목과 같은 학습할 때 방해가 되는 데이터를 먼저 솎아냅시다. 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해보세요.

**✓ 중복 샘플과 NULL 값이 존재하는 샘플 제거 : 데이터의 중복 샘플 유무를 확인**

In [11]:
print('text 열에서 중복을 배제한 유일한 샘플의 수 :', data['text'].nunique(),'개')
print('headlines 열에서 중복을 배제한 유일한 샘플의 수 :', data['headlines'].nunique(),'개')

text 열에서 중복을 배제한 유일한 샘플의 수 : 98360 개
headlines 열에서 중복을 배제한 유일한 샘플의 수 : 98280 개


In [16]:
# 데이터프레임의 drop_duplicates()를 사용하면, 손쉽게 중복 샘플을 제거
data.drop_duplicates(subset = ['text'], inplace = True)

print('전체 샘플수 :',(len(data)),'개')

전체 샘플수 : 98360 개


In [13]:
print(data.isnull().sum())  # 이터프레임에 Null 값이 있는지 확인하는 방법

text         0
headlines    0
dtype: int64


In [18]:
# 데이터프레임에서 Null을 제거할 때는 dropna() 함수를 사용
data.dropna(axis = 0, inplace = True)
print('전체 샘플수 :',(len(data)),'개')

전체 샘플수 : 98360 개


**✓ 텍스트 정규화와 불용어 제거**

In [ ]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

print("정규화 사전의 수: ",len(contractions),'개')

In [ ]:
print('불용어 개수 :', len(stopwords.words('english') ))
print(stopwords.words('english'))

### <span style="color:green">Step3. 어텐션 메커니즘 사용하기 (추상적 요약)</span>
---
일반적인 seq2seq보다는 어텐션 메커니즘을 사용한 seq2seq를 사용하는 것이 더 나은 성능을 얻을 수 있어요. 실습 내용을 참고하여 어텐션 메커니즘을 사용한 seq2seq를 설계해 보세요.

### <span style="color:green">Step4. 실제 결과와 요약문 비교하기 (추상적 요약)</span>
---
실습에서 사용된 전처리를 참고하여 각자 필요하다고 생각하는 전처리를 추가 사용하여 텍스트를 정규화 또는 정제해 보세요. 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해보세요.

### <span style="color:green">Step5. Summa을 이용해서 추출적 요약해보기</span>
---
추상적 요약은 추출적 요약과는 달리 문장의 표현력을 다양하게 가져갈 수 있지만, 추출적 요약에 비해서 난이도가 높아요. 반대로 말하면 추출적 요약은 추상적 요약에 비해 난이도가 낮고 기존 문장에서 문장을 꺼내오는 것이므로 잘못된 요약이 나올 가능성이 낮아요.

Summa의 summarize를 사용하여 추출적 요약을 해보세요.

# <span style="color:purple">루브릭 평가 기준</span>
---
번호|평가문항|상세기준
---|---|---
1 |Abstractive 모델 구성을 위한 텍스트 전처리 단계가 체계적으로 진행되었다.|분석단계, 정제단계, 정규화와 불용어 제거, 데이터셋 분리, 인코딩 과정이 빠짐없이 체계적으로 진행되었다.
2 |텍스트 요약모델이 성공적으로 학습되었음을 확인하였다.|모델학습이 안정적으로 수렴되었음을 그래프를 통해 확인하였으며, 실제 요약문과 유사한 요약문장을 얻을 수 있었다.
3 |Extractive 요약을 시도해 보고 Abstractive 요약 결과과 함께 비교해 보았다.|두 요약 결과를 문법완성도 측면과 핵심단어 포함 측면으로 나누어 비교분석 결과를 제시하였다.

# <span style="color:purple">회고</span>
---